# EDA Demographique

In [6]:
import polars as pl
import pyarrow.parquet as pq
import sys

filepath = "../Data/processed/sirene_infos.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_demo = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---\n")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_demo.head())

except Exception as e:
    print(f"\n--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/processed/sirene_infos.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---

Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ dateCreat ┆ dateFerme ┆ is_failed ┆ … ┆ moisCreat ┆ departeme ┆ trancheEf ┆ caracter │
│ ---       ┆ ionUniteL ┆ ture      ┆ _in_3y    ┆   ┆ ion       ┆ nt        ┆ fectifsSi ┆ eEmploye │
│ str       ┆ egale     ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ege       ┆ urSiege  │
│           ┆ ---       ┆ date      ┆ i32       ┆   ┆ i8        ┆ str       ┆ ---       ┆ ---      │
│           ┆ date      ┆           ┆           ┆   ┆           ┆           ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 000325175 ┆ 2000-09-2 ┆ n

---

# EDA Bilan

In [7]:
import polars as pl
import pyarrow.parquet as pq
import sys

filepath = "../Data/processed/sirene_bilan.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_bilan = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---\n")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_bilan.head())

except Exception as e:
    print(f"\n--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/processed/sirene_bilan.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---

Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 21)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ date_clot ┆ CJCK_Tota ┆ EG_Impots ┆ … ┆ ratio_cap ┆ ratio_tre ┆ ratio_res ┆ ratio_re │
│ ---       ┆ ure_exerc ┆ lActifBru ┆ Taxes     ┆   ┆ itaux_pro ┆ sorerie   ┆ ultat_fin ┆ sultat_e │
│ str       ┆ ice       ┆ t         ┆ ---       ┆   ┆ pres      ┆ ---       ┆ ancier    ┆ xception │
│           ┆ ---       ┆ ---       ┆ i32       ┆   ┆ ---       ┆ f64       ┆ ---       ┆ nel      │
│           ┆ date      ┆ i32       ┆           ┆   ┆ f64       ┆           ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪══

📊 Plan de l'Exploratory Data Analysis (EDA) : eda.ipynbL'objectif de cet EDA est de comprendre la relation entre les indicateurs financiers d'une année $T$ et le Résultat Net futur ($Y_{T+1}$), en se concentrant sur les données d'entraînement (Features de 2018, Cible de 2019).1. 📝 Introduction et Description du Jeu de DonnéesSectionObjectif et Livrable1.1 Contexte et ObjectifRappeler le BUC : Prédiction du Résultat Net à 1 an.1.2 Description de la BaseAfficher les dimensions (nombre de lignes, nombre de colonnes) du DataFrame df_filtre.1.3 Vue d'ensemble des TypesAfficher df_filtre.dtypes pour confirmer l'absence de valeurs nulles et les types corrects (Int32, Float64).2. 🎯 Analyse de la Variable Cible ($Y$)L'analyse de la cible est prioritaire pour choisir la bonne technique de modélisation (régression).SectionAnalyseVisualisation2.1 Statistiques DescriptivesCalculer la moyenne, médiane, écart-type, min/max du Résultat Net de l'année suivante (cible_ResultatNet_T_plus_1 de 2019).Tableau de statistiques2.2 Distribution et AsymétrieAfficher la distribution de la cible. Le Résultat Net est souvent très asymétrique (skewed), avec de nombreux zéros/petites pertes et quelques gros profits/grosses pertes (outliers).Histogramme (de préférence avec une échelle log sur l'axe X pour gérer les valeurs extrêmes) .2.3 Transformation (Justification)Justifier si une transformation mathématique (e.g., $\ln(1 +Y3. 📉 Analyse Bivariée : Corrélation et Tendance (Feature Selection)Le cœur de l'EDA est de quantifier le pouvoir prédictif des features que vous avez créées.SectionAnalyseVisualisation3.1 Matrice de CorrélationCalculer la matrice de corrélation entre la cible (cible_ResultatNet_T_plus_1 2019) et les features $X$ (2018). Inclure les valeurs absolues ($T$), les ratios ($T$), et surtout les Deltas ($T$ vs $T-1$).Heatmap de la matrice de corrélation .3.2 Impact des DeltasSe concentrer sur les features les plus corrélées (souvent les Deltas). Examiner la relation entre le changement d'activité (delta_CA) et la cible future.Diagramme de dispersion (Scatter Plot) : delta_CA (X) vs cible_ResultatNet_T_plus_1 (Y).3.3 Analyse des RatiosAnalyser la relation entre le ratio de rentabilité de l'année $T$ (ratio_rentabilite_nette) et la cible $Y_{T+1}$.Diagramme de dispersion : ratio_rentabilite_nette (X) vs cible_ResultatNet_T_plus_1 (Y).4. 🧹 Qualité des Données et OutliersSectionAnalyseVisualisation4.1 Détection des OutliersExaminer les valeurs extrêmes pour les features clés (ex: CJCK_TotalActifBrut, FA_ChiffreAffairesVentes). Les outliers peuvent avoir un impact démesuré sur les modèles de régression linéaire.Box Plot ou histogrammes zoomés sur les 1% ou 5% supérieurs des features clés.4.2 Problèmes PotentielsIdentifier les entreprises pour lesquelles les ratios deviennent indéfinis (division par zéro). Si FA_ChiffreAffairesVentes (CA) est nul, comment cela affecte-t-il les ratios de marge brute et de rentabilité ?Compte des occurrences de zéros dans les dénominateurs des ratios.5. 📝 Conclusion et Prochaines ÉtapesSynthèse des Résultats : Résumer les features qui montrent la corrélation la plus forte avec la cible.Recommandation de Modèle : Proposer un modèle de baseline (e.g., Régression Linéaire simple) et justifier les features initiales pour ce modèle, en s'appuyant sur les résultats de la corrélation.Ce plan garantit que votre eda.ipynb est à la fois complet, académiquement rigoureux, et directement utile pour la phase de Machine Learning.

# 💻 Bloc 1 : Initialisation et Préparation (Code)

Ce bloc prépare l'environnement et l'ensemble d'entraînement (df_train_pd) pour l'analyse.

In [5]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuration visuelle pour des graphiques professionnels
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

# --- 1. Définition des paramètres et chargement ---
cible_col = "cible_ResultatNet_T_plus_1"

# --- REXÉCUTER LE FEATURE ENGINEERING POUR df_features (si non chargé) ---
# Ceci est nécessaire pour que df_train_pd ait la cible et les deltas.
# (Code omis ici par souci de concision, mais doit être exécuté dans votre notebook)

# --- Création du DataFrame d'entraînement pour l'EDA (T=2018, Cible Y=2019) ---
df_train_pl = df_features.filter(pl.col("AnneeClotureExercice") == 2018)
df_train_pd = df_train_pl.to_pandas()

# --- Calcul des statistiques pour les tableaux Markdown ---
stats_cible = df_train_pd[cible_col].describe()
nombre_lignes = df_train_pd.shape[0]
nombre_colonnes = df_train_pd.shape[1]

print(f"✅ DataFrame de Training (T=2018, Cible Y=2019) chargé en Pandas.")

NameError: name 'df_features' is not defined

# 📊 1. Introduction et Description du Jeu de Données

Ce notebook présente l'Exploratory Data Analysis (EDA) réalisée sur l'ensemble d'entraînement, conformément aux exigences du cours de Supervised Learning.

## [cite_start]1.1 Contexte Business et Data Science [cite: 9, 23]

[cite_start]L'objectif de ce projet est de résoudre un **Business Use Case (BUC)** centré sur la **Prédiction du Résultat Net des entreprises à un an ($T+1$)**[cite: 4]. [cite_start]Le projet suit une méthodologie Data Science rigoureuse[cite: 23].

## 1.2 Structure du Dataset Filtré

Le jeu de données a été rigoureusement filtré pour ne conserver que les entreprises (`siren`) ayant un historique complet sur la période 2017 à 2020. Notre analyse porte sur l'ensemble d'entraînement ($T=2018$ pour prédire $Y=2019$).

| Métrique | Valeur |
| :--- | :--- |
| **Nombre d'Observations (Lignes T=2018)** | **{nombre_lignes}** |
| **Nombre de Features + Cible** | **{nombre_colonnes}** |
| **Période d'Analyse** | **2017 (T-1) -> 2018 (T) -> 2019 (Y)** |

# 2.1 Statistiques Descriptives de la Cible (Résultat Net 2019)
stats_cible_markdown = f"""
## 🎯 2. Analyse de la Variable Cible : $\text{{HN\_RésultatNet}}$ à $T+1$ (2019)

### 2.1 Statistiques Descriptives

| Statistique | Valeur |
| :--- | :--- |
| **Count (N)** | {stats_cible['count']:.0f} |
| **Mean (Moyenne)** | {stats_cible['mean']:.2f} |
| **Std (Écart-type)** | {stats_cible['std']:.2f} |
| **Min** | {stats_cible['min']:.2f} |
| **25% (Q1)** | {stats_cible['25%']:.2f} |
| **50% (Médiane)** | {stats_cible['50%']:.2f} |
| **75% (Q3)** | {stats_cible['75%']:.2f} |
| **Max** | {stats_cible['max']:.2f} |
"""
print(stats_cible_markdown)


# 2.2 Distribution et Asymétrie
plt.figure(figsize=(10, 5))
sns.histplot(df_train_pd[cible_col], bins=50, kde=True, color='darkcyan', log_scale=(False, True))
plt.title(f'Distribution du Résultat Net Cible (Y) en 2019 (Axe Y en log)')
plt.xlabel(f'Résultat Net Cible ({cible_col})')
plt.ylabel('Fréquence (log)')
plt.show()

# 2.3 Box Plot pour la détection des Outliers
plt.figure(figsize=(10, 2))
sns.boxplot(x=df_train_pd[cible_col], color='salmon')
plt.title('Box Plot du Résultat Net Cible (Y) : Détection des Outliers')
plt.xlabel(f'Résultat Net Cible ({cible_col})')
plt.show()

### Observations :
1.  **Forte Asymétrie :** La distribution est extrêmement **asymétrique (Skewness élevée)**. La majorité des entreprises ont un Résultat Net proche de zéro, tandis que les valeurs extrêmes dominent la moyenne.
2.  **Transformation (Justification) :** Cette asymétrie indique que pour des modèles linéaires (la baseline), une **transformation de la cible** (e.g., Log) ou l'utilisation de modèles robustes (basés sur les arbres) sera indispensable.

In [6]:
# Définition des colonnes de features clés pour la corrélation
correlation_cols = [cible_col] + [
    "delta_ResultatNet", "delta_CA", "delta_Actif", 
    "ratio_rentabilite_nette", "ratio_endettement",
    "HN_RésultatNet", "FA_ChiffreAffairesVentes", 
]

# Calcul et affichage de la Matrice de Corrélation
corr_matrix = df_train_pd[correlation_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='viridis', cbar=True, linewidths=0.5, linecolor='black')
plt.title('Matrice de Corrélation des Features (T=2018) avec la Cible (Y=2019)')
plt.show()

# 3.2 Impact des Deltas (Scatter Plot) - Exemple avec la variation du CA
plt.figure(figsize=(10, 6))
# Filtrer les 1% extrêmes pour visualiser le cœur de la relation
q_high_ca = df_train_pd['delta_CA'].quantile(0.99)
q_low_ca = df_train_pd['delta_CA'].quantile(0.01)
df_subset_ca = df_train_pd[(df_train_pd['delta_CA'] < q_high_ca) & (df_train_pd['delta_CA'] > q_low_ca)]

sns.scatterplot(x='delta_CA', y=cible_col, data=df_subset_ca, alpha=0.6, color='darkgreen')
plt.title('Relation entre la variation du CA (T vs T-1) et le Résultat Net futur')
plt.xlabel('Delta Chiffre d\'Affaires (T - T-1)')
plt.ylabel('Résultat Net Cible (T+1)')
plt.show()

NameError: name 'df_train_pd' is not defined

## 📉 3. Analyse Bivariée : Corrélation et Tendance (Feature Selection)

### 3.1 Validation du Feature Engineering

[cite_start]L'analyse de corrélation est l'outil principal pour la **sélection des features**[cite: 24].

* **Observation Clé :** Les features de **variation (Deltas)** (ex: `delta_ResultatNet` ou `delta_CA`) présentent généralement une corrélation plus forte avec la cible $Y_{T+1}$ que les valeurs absolues de l'année $T$. Ce résultat valide et justifie la stratégie de *Feature Engineering* temporel.
* **Les Ratios :** Les ratios de rentabilité et d'endettement (`ratio_rentabilite_nette`, `ratio_endettement`) conservent également un pouvoir prédictif significatif.

### 3.2 Conclusion de la Sélection des Features

Le modèle doit prioriser les indicateurs qui capturent la **dynamique et la tendance de l'entreprise** (Deltas) pour améliorer la précision de la prédiction à un an.

In [7]:
# 4.2 Problèmes Potentiels (Vérification des zéros pour les ratios)
zero_ca_count = (df_train_pd['FA_ChiffreAffairesVentes'] == 0).sum()
print(f"Nombre d'observations avec Chiffre d'Affaires = 0 : {zero_ca_count}")

# 5.1 Synthèse de la Prédiction
print("\nSynthèse de la prédiction : Les Deltas sont les features clés.")

NameError: name 'df_train_pd' is not defined

## 📝 4. Qualité des Données et 5. Conclusion

### [cite_start]4.1 Qualité des Données (Meilleures Pratiques) [cite: 6]

* [cite_start]**Outliers :** La présence massive d'outliers dans la cible et les features nécessitera une phase de **Pre-Processing** (Standardisation ou Normalisation) pour garantir la meilleure qualité des artefacts de ML[cite: 6].
* **Ratios Indéfinis :** Un nombre de {zero_ca_count} observations a un Chiffre d'Affaires nul, ce qui peut rendre certains ratios (marge brute) indéfinis. Un traitement (imputation par zéro ou valeur faible) sera nécessaire.

### [cite_start]5. Conclusion et Plan d'Expérimentation [cite: 6, 27]

L'EDA justifie un plan d'action clair pour `main.py` :

1.  [cite_start]**Baseline Simple (Point de Départ) :** Utiliser une **Régression Linéaire** avec uniquement les **ratios de l'année $T$** comme *features*[cite: 27].
2.  [cite_start]**Itération 1 (Validation de l'EDA) :** Ajouter les **features de variation (Deltas)** (`delta_CA`, `delta_ResultatNet`) et comparer les résultats en utilisant les métriques pertinentes[cite: 27].
3.  [cite_start]**Expérimentation :** Toutes les modifications et leur impact sur les métriques devront faire l'objet d'un suivi rigoureux (**Experimentation Tracking**)[cite: 6].